In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_met = pd.read_csv('../data/xls_Bulath/Bullath_metadata.csv',sep = '\t')
df_smb      = pd.read_excel('../data/xls_Bulath/Bullath_all.xls', header=None)
smb_data    = df_smb.values[:,1:-1]
smb_time    = df_smb.values[:,0]
df_smb.columns = df_met.Name_orig

In [5]:
smb_coll = []
for i in df_met.Name_orig[1:]:
    subset = (df_smb[['Date', i]])
    subset.rename(columns = {i:'SMB'},  inplace=True)
    subset['Stake'] = i
    g_met = df_met.loc[lambda df_met: df_met['Name_orig'] == i, :]
    subset['Latitude']  = float(g_met.Latitude.values)
    subset['Longitude'] = float(g_met.Longitude.values)
    subset['Elevation'] = float(g_met.Elevation.values)
    subset.dropna(inplace=True)
    subset = subset[['Stake', 'Date', 'Latitude', 'Longitude', 'Elevation', 'SMB']]
    subset['SMB'] = subset['SMB']/100
    smb_coll.append(subset)

In [6]:
smb_coll_df = pd.concat(smb_coll, ignore_index=True)

In [7]:
smb_coll_df

Name_orig,Stake,Date,Latitude,Longitude,Elevation,SMB
0,R1,2007-11-08,-62.177620,-58.902406,82.0,0.68800
1,R1,2007-12-09,-62.177620,-58.902406,82.0,0.75840
2,R1,2007-12-18,-62.177620,-58.902406,82.0,0.73730
3,R1,2007-12-26,-62.177620,-58.902406,82.0,0.65472
4,R1,2008-01-05,-62.177620,-58.902406,82.0,0.44000
...,...,...,...,...,...,...
2560,JN,2012-02-04,-62.152849,-58.877932,231.0,0.12325
2561,JN,2012-02-19,-62.152849,-58.877932,231.0,0.03928
2562,JN,2012-03-04,-62.152849,-58.877932,231.0,-0.03280
2563,JN,2012-03-17,-62.152849,-58.877932,231.0,-0.18860


In [7]:
df_s = smb_coll_df.sort_values(by=['Elevation', 'Date'])
df_s.index = df_s.Date
df_s['Year'] = df_s.index.year

In [8]:
labels = df_s.drop_duplicates(subset=['Stake'])

In [9]:
label_year = df_s.drop_duplicates(subset=['Year']).sort_values(by=['Year'])['Year'].values
label_year[5]

2012

In [10]:
smb_sea = []
for j in labels['Stake'][0:]:
    stake = df_s.loc[lambda df: df['Stake'] == j, :]
    
    for i in label_year[:-1]:
        # SMB winter
        time1_w = str(i)+'0401'
        time2_w = str(i+1)+'0305'
        subset_w = stake[time1_w:time2_w]
        smb_w = subset_w.loc[subset_w['SMB'] == subset_w['SMB'].max()]
        if len(smb_w) > 1:
            smb_w = smb_w.loc[[smb_w.index[0]]]
        smb_sea.append(smb_w)

        # SMB summer
        time1_s = str(i)+'1201'
        time2_s = str(i+1)+'0331'
        subset_s =stake[time1_s:time2_s]
        smb_s = subset_s.loc[subset_s['SMB'] == subset_s['SMB'].min()]
        if len(smb_s) > 1:
            smb_s = smb_s.loc[[smb_s.index[-1]]]
        smb_sea.append(smb_s)

In [11]:
smb_sea_t = pd.concat(smb_sea)
smb_sea_t.reset_index(drop=True, inplace=True)

In [12]:
smb_sea_t0 = []
for j in labels['Stake'][0:]:
    stake = smb_sea_t.loc[lambda df: df['Stake'] == j, :].copy()
    date0 = stake[0:1].copy()
    date0['SMB'] = 0
    if date0['Year'].values == 2007:
        date0['Date'] = np.datetime64('2007-05-01')
    else:
        date0['Date'] = np.datetime64('2007-05-01')
    smb_sea_t0.append(date0)
    smb_sea_t0.append(stake)

In [13]:
smb_sea_t = pd.concat(smb_sea_t0)

In [14]:
date_1 = []
for j in labels['Stake'][0:]:    
    stake = smb_sea_t.loc[lambda df: df['Stake'] == j, :].copy()
    date_1.append(stake['Date'].values[0])
    for i in stake.Date[0:-1]:
        date_1.append(i)

In [15]:
smb_sea_t['Date1'] = date_1

In [16]:
smb_sea_t['Days'] = (smb_sea_t['Date'] - smb_sea_t['Date1']).dt.days

In [17]:
smb_coll_df1 = smb_sea_t[['Stake', 'Date', 'Date1', 'Latitude', 'Longitude', 
                          'Elevation', 'Days','SMB']]

In [18]:
smb_coll_df1

Name_orig,Stake,Date,Date1,Latitude,Longitude,Elevation,Days,SMB
0,R15,2007-05-01,2007-05-01,-62.178873,-58.907921,54.0,0,0.00000
0,R15,2008-01-05,2007-05-01,-62.178873,-58.907921,54.0,249,0.49203
1,R15,2008-03-11,2008-01-05,-62.178873,-58.907921,54.0,66,-0.46800
2,R15,2009-01-24,2008-03-11,-62.178873,-58.907921,54.0,319,0.00000
3,R15,2009-03-01,2009-01-24,-62.178873,-58.907921,54.0,36,-0.66600
...,...,...,...,...,...,...,...,...
283,9N,2010-03-23,2009-12-07,-62.166223,-58.883543,261.0,106,0.33540
284,9N,2010-12-30,2010-03-23,-62.166223,-58.883543,261.0,282,1.22980
285,9N,2011-03-22,2010-12-30,-62.166223,-58.883543,261.0,82,0.53400
286,9N,2011-12-04,2011-03-22,-62.166223,-58.883543,261.0,257,1.17298


In [19]:
smb_coll_df1.to_csv('../data/SMB_bellin_season.csv', index=False, sep='\t')